In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import timedelta
from scipy.signal import argrelextrema
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class GoldChartGenerator:
    def __init__(self, symbol="GC=F"):
        self.symbol = symbol
        self.df = None
    
    def fetch_data(self, period="5d", interval="5m"):
        print(f"--- {self.symbol} データ取得中 ---")
        try:
            df = yf.download(self.symbol, interval=interval, period=period, progress=False)
            if df.empty: return None
            
            if isinstance(df.columns, pd.MultiIndex): 
                df.columns = df.columns.get_level_values(0)
            
            for col in ["Open", "High", "Low", "Close", "Volume"]:
                df[col] = pd.to_numeric(df[col], errors="coerce")
            
            df.dropna(inplace=True)
            if df.index.tz is None: 
                df.index = df.index.tz_localize('UTC')
            df.index = df.index.tz_convert('Asia/Tokyo')
            df.index.name = "Datetime"
            
            df['MA200'] = df['Close'].rolling(window=200).mean()
            self.df = df
            return df
        except Exception as e:
            print(f"データ取得エラー: {e}")
            return None

    def _get_yesterday_levels(self):
        if self.df is None: return None, None
        last_date = self.df.index[-1].date()
        prev_data = self.df[self.df.index.date < last_date]
        if prev_data.empty: return None, None
        
        target_date = prev_data.index[-1].date()
        target = prev_data[prev_data.index.date == target_date]
        return target['High'].max(), target['Low'].min()

    def _get_strong_levels(self, df, window=15, merge_dist=2.5, min_touches=3):
        levels = []
        for col, comp in [('High', np.greater), ('Low', np.less)]:
            vals = df[col].values
            idxs = argrelextrema(vals, comparator=comp, order=window)[0]
            for idx in idxs:
                levels.append(df.iloc[idx][col])
        
        levels.sort()
        if not levels: return []

        clusters = []
        current_cluster = [levels[0]]
        for i in range(1, len(levels)):
            if levels[i] - current_cluster[-1] < merge_dist:
                current_cluster.append(levels[i])
            else:
                clusters.append(current_cluster)
                current_cluster = [levels[i]]
        clusters.append(current_cluster)

        strong_zones = []
        for cluster in clusters:
            if len(cluster) >= min_touches:
                strong_zones.append(np.mean(cluster))
        return strong_zones

    def generate_chart(self, display_hours=8):
        if self.df is None:
            print("データがありません")
            return
        
        # 表示範囲
        plot_start = self.df.index[-1] - timedelta(hours=display_hours)
        df_plot = self.df[self.df.index >= plot_start].copy()
        if df_plot.empty: return

        current_price = float(self.df['Close'].iloc[-1])
        
        # サポレジ取得
        y_high, y_low = self._get_yesterday_levels()
        levels = self._get_strong_levels(self.df, window=15, merge_dist=2.5, min_touches=3)
        
        res_levels = sorted([x for x in levels if x > current_price])
        sup_levels = sorted([x for x in levels if x < current_price], reverse=True)
        
        # --- Plotly 描画開始 ---
        
        # レイアウト：2段（チャート80%, 出来高20%）
        fig = make_subplots(
            rows=2, cols=1, 
            shared_xaxes=True, 
            vertical_spacing=0.03, 
            row_heights=[0.8, 0.2]
        )

        # 1. ローソク足
        fig.add_trace(go.Candlestick(
            x=df_plot.index,
            open=df_plot['Open'], high=df_plot['High'],
            low=df_plot['Low'], close=df_plot['Close'],
            name='Price',
            increasing_line_color='#00ff00', decreasing_line_color='#ff0000'
        ), row=1, col=1)

        # 2. MA200
        fig.add_trace(go.Scatter(
            x=df_plot.index, y=df_plot['MA200'],
            mode='lines', name='MA200',
            line=dict(color='orange', width=1.5)
        ), row=1, col=1)

        # 3. ボリューム
        colors = ['#00ff00' if c >= o else '#ff0000' for c, o in zip(df_plot['Close'], df_plot['Open'])]
        fig.add_trace(go.Bar(
            x=df_plot.index, y=df_plot['Volume'],
            name='Volume', marker_color=colors, opacity=0.3
        ), row=2, col=1)

        # --- 水平線・帯（ゾーン）の描画 ---
        NEON_COLORS = ['#FFD700', '#00FFFF', '#FF00FF', '#ADFF2F', '#1E90FF']
        
        # ヘルパー関数: 線と帯を追加
        def add_zone_level(level, color, style, width, line_opacity, zone_opacity, name):
            # 1. 帯（背景の塗りつぶし）: add_hrectを使用
            # レートの上下に少し幅を持たせる (例: ±1.5ドル)
            zone_width = 1.5 
            fig.add_hrect(
                y0=level - zone_width, 
                y1=level + zone_width,
                fillcolor=color, 
                opacity=zone_opacity,
                layer="below", # ローソク足の下に描画
                line_width=0,
                row=1, col=1
            )
            
            # 2. メインの線: add_hlineを使用
            fig.add_hline(
                y=level,
                line_dash=style,
                line_color=color,
                line_width=width,
                opacity=line_opacity,
                row=1, col=1,
                annotation_text=name, # 線の端にラベルを表示（オプション）
                annotation_position="top right",
                annotation_font_size=10,
                annotation_font_color=color
            )

        # 昨日の高安（赤・青）
        if y_high: add_zone_level(y_high, 'red', 'solid', 2, 1.0, 0.15, "Prev High")
        if y_low: add_zone_level(y_low, 'blue', 'solid', 2, 1.0, 0.15, "Prev Low")

        # レジスタンス
        for i, level in enumerate(res_levels):
            is_nearest = (i == 0)
            color = '#DC143C' if is_nearest else NEON_COLORS[i % len(NEON_COLORS)]
            style = 'dash'
            width = 2 if is_nearest else 1
            l_opacity = 1.0 if is_nearest else 0.8
            z_opacity = 0.2 if is_nearest else 0.1 # 直近は帯を少し濃く
            
            add_zone_level(level, color, style, width, l_opacity, z_opacity, "")

        # サポート
        for i, level in enumerate(sup_levels):
            is_nearest = (i == 0)
            color = '#DC143C' if is_nearest else NEON_COLORS[(i + 2) % len(NEON_COLORS)]
            style = 'dot'
            width = 2 if is_nearest else 1
            l_opacity = 1.0 if is_nearest else 0.8
            z_opacity = 0.2 if is_nearest else 0.1
            
            add_zone_level(level, color, style, width, l_opacity, z_opacity, "")

        # --- レイアウト設定 ---
        fig.update_layout(
            title=dict(
                text=f"GOLD (XAU/USD) - {current_price:.2f}",
                font=dict(size=20, color="white")
            ),
            template="plotly_dark",
            xaxis_rangeslider_visible=False,
            height=700,
            showlegend=False,
            margin=dict(l=20, r=20, t=60, b=20),
            # Y軸の自動調整を少し緩めに（帯が見えるように）
            yaxis=dict(side="right") # FXチャートっぽく右側に目盛り
        )
        
        # 凡例テキスト
        fig.add_annotation(
            xref="paper", yref="paper",
            x=0.01, y=0.99,
            text="<b>Zone Legend</b><br>■ Red: Prev High<br>■ Blue: Prev Low<br>■ Crimson: Nearest S/R",
            showarrow=False,
            font=dict(color="white", size=10),
            align="left",
            bgcolor="rgba(0,0,0,0.6)",
            bordercolor="gray"
        )

        fig.show()

if __name__ == "__main__":
    gen = GoldChartGenerator()
    df = gen.fetch_data(period="5d", interval="5m")
    if df is not None:
        gen.generate_chart(display_hours=12)

--- GC=F データ取得中 ---


In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import timedelta
from scipy.signal import argrelextrema
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class GoldChartGenerator:
    def __init__(self, symbol="GC=F"):
        self.symbol = symbol
        self.df = None
    
    def fetch_data(self, period="5d", interval="5m"):
        print(f"--- {self.symbol} データ取得中 ---")
        try:
            df = yf.download(self.symbol, interval=interval, period=period, progress=False)
            if df.empty: return None
            
            if isinstance(df.columns, pd.MultiIndex): 
                df.columns = df.columns.get_level_values(0)
            
            for col in ["Open", "High", "Low", "Close", "Volume"]:
                df[col] = pd.to_numeric(df[col], errors="coerce")
            
            df.dropna(inplace=True)
            if df.index.tz is None: 
                df.index = df.index.tz_localize('UTC')
            df.index = df.index.tz_convert('Asia/Tokyo')
            df.index.name = "Datetime"
            
            df['MA200'] = df['Close'].rolling(window=200).mean()
            self.df = df
            return df
        except Exception as e:
            print(f"データ取得エラー: {e}")
            return None

    def _get_yesterday_levels(self):
        if self.df is None: return None, None
        last_date = self.df.index[-1].date()
        prev_data = self.df[self.df.index.date < last_date]
        if prev_data.empty: return None, None
        
        target_date = prev_data.index[-1].date()
        target = prev_data[prev_data.index.date == target_date]
        return target['High'].max(), target['Low'].min()

    def _get_strong_levels(self, df, window=5, merge_dist=2.5, min_touches=2):
        """
        修正: merge_dist を 4.0 -> 2.5 へ変更
        （まとめすぎず、適度に近いラインも残す）
        """
        levels = []
        for col, comp in [('High', np.greater), ('Low', np.less)]:
            vals = df[col].values
            idxs = argrelextrema(vals, comparator=comp, order=window)[0]
            for idx in idxs:
                levels.append(df.iloc[idx][col])
        
        # 期間内の最高値・最安値
        levels.append(df['High'].max())
        levels.append(df['Low'].min())
        
        levels.sort()
        if not levels: return []

        clusters = []
        current_cluster = [levels[0]]
        for i in range(1, len(levels)):
            if levels[i] - current_cluster[-1] < merge_dist:
                current_cluster.append(levels[i])
            else:
                clusters.append(current_cluster)
                current_cluster = [levels[i]]
        clusters.append(current_cluster)

        strong_zones = []
        for cluster in clusters:
            avg_level = np.mean(cluster)
            is_extreme = (np.isclose(avg_level, df['High'].max(), atol=1.5) or 
                          np.isclose(avg_level, df['Low'].min(), atol=1.5))
            
            if len(cluster) >= min_touches or is_extreme:
                strong_zones.append(avg_level)
                
        return strong_zones

    def generate_chart(self, display_hours=8):
        if self.df is None:
            print("データがありません")
            return
        
        plot_start = self.df.index[-1] - timedelta(hours=display_hours)
        df_plot = self.df[self.df.index >= plot_start].copy()
        if df_plot.empty: return

        current_price = float(self.df['Close'].iloc[-1])
        
        # サポレジ取得（マージ距離2.5）
        y_high, y_low = self._get_yesterday_levels()
        levels = self._get_strong_levels(self.df, window=5, merge_dist=2.5, min_touches=2)
        
        levels = sorted(list(set(levels)))

        # フィルター: 現在価格から±2.0%（少し広めに見る）
        threshold = current_price * 0.02 
        relevant_levels = [x for x in levels if abs(x - current_price) < threshold]

        res_levels = [x for x in relevant_levels if x > current_price]
        sup_levels = [x for x in relevant_levels if x < current_price]
        
        sup_levels.sort(reverse=True)
        res_levels.sort()
        
        # --- Plotly Drawing ---
        fig = make_subplots(
            rows=2, cols=1, shared_xaxes=True, 
            vertical_spacing=0.03, row_heights=[0.8, 0.2]
        )

        # 1. Price
        fig.add_trace(go.Candlestick(
            x=df_plot.index,
            open=df_plot['Open'], high=df_plot['High'],
            low=df_plot['Low'], close=df_plot['Close'],
            name='Price',
            increasing_line_color='#00ff00', decreasing_line_color='#ff0000'
        ), row=1, col=1)

        # 2. MA200
        fig.add_trace(go.Scatter(
            x=df_plot.index, y=df_plot['MA200'],
            mode='lines', name='MA200',
            line=dict(color='orange', width=1.5)
        ), row=1, col=1)

        # 3. Volume
        colors = ['#00ff00' if c >= o else '#ff0000' for c, o in zip(df_plot['Close'], df_plot['Open'])]
        fig.add_trace(go.Bar(
            x=df_plot.index, y=df_plot['Volume'],
            name='Volume', marker_color=colors, opacity=0.3
        ), row=2, col=1)

        # --- Zone Helper ---
        NEON_COLORS = ['#FFD700', '#00FFFF', '#FF00FF', '#ADFF2F', '#1E90FF']
        
        def add_zone_level(level, color, style, width, l_opacity, z_opacity, name):
            # ゾーン幅設定
            zone_width = 1.5
            
            # 帯
            fig.add_hrect(
                y0=level - zone_width, y1=level + zone_width,
                fillcolor=color, opacity=z_opacity, layer="below", line_width=0,
                row=1, col=1
            )
            # 線
            fig.add_hline(
                y=level, line_dash=style, line_color=color, line_width=width, opacity=l_opacity,
                row=1, col=1, annotation_text=name, annotation_position="top right",
                annotation_font=dict(size=10, color=color)
            )

        # --- 昨日の高安（最重要・常に表示）---
        if y_high: add_zone_level(y_high, 'red', 'solid', 3, 0.9, 0.15, f"Prev High {y_high:.1f}")
        if y_low: add_zone_level(y_low, 'blue', 'solid', 3, 0.9, 0.15, f"Prev Low {y_low:.1f}")

        # --- レジスタンス描画ロジック ---
        # 直近3本まではハッキリ色分けして表示する
        for i, level in enumerate(res_levels):
            if i == 0:
                # 直近：エントリー用（赤・Crimson）
                color = '#DC143C'
                style = 'dash'
                width = 2
                l_op, z_op = 1.0, 0.2
                label = f"Res 1 (Nearest) {level:.1f}"
            elif i <= 2:
                # 2本目・3本目：警戒ゾーン（ネオンカラー）
                color = NEON_COLORS[i % len(NEON_COLORS)]
                style = 'dot'
                width = 1
                l_op, z_op = 0.8, 0.1
                label = f"Res {i+1} {level:.1f}"
            else:
                # 4本目以降：薄く（背景情報）
                color = 'rgba(255, 215, 0, 0.3)'
                style = 'dot'
                width = 1
                l_op, z_op = 0.4, 0.05
                label = "" # うるさいのでラベルなし

            add_zone_level(level, color, style, width, l_op, z_op, label)

        # --- サポート描画ロジック ---
        for i, level in enumerate(sup_levels):
            if i == 0:
                # 直近：エントリー用（青・DeepSkyBlue）
                color = '#00BFFF'
                style = 'dash'
                width = 2
                l_op, z_op = 1.0, 0.2
                label = f"Sup 1 (Nearest) {level:.1f}"
            elif i <= 2:
                # 2本目・3本目：警戒ゾーン
                color = NEON_COLORS[(i + 2) % len(NEON_COLORS)]
                style = 'dot'
                width = 1
                l_op, z_op = 0.8, 0.1
                label = f"Sup {i+1} {level:.1f}"
            else:
                # 4本目以降
                color = 'rgba(255, 215, 0, 0.3)'
                style = 'dot'
                width = 1
                l_op, z_op = 0.4, 0.05
                label = ""

            add_zone_level(level, color, style, width, l_op, z_op, label)

        # Layout
        fig.update_layout(
            title=dict(text=f"GOLD (XAU/USD) - Tactical View : {current_price:.2f}", font=dict(size=20, color="white")),
            template="plotly_dark",
            xaxis_rangeslider_visible=False,
            height=750,
            showlegend=False,
            margin=dict(l=50, r=50, t=60, b=20),
            yaxis=dict(side="right")
        )
        
        # ガイドテキスト修正
        fig.add_annotation(
            xref="paper", yref="paper",
            x=0.01, y=0.99,
            text="<b>Tactical View</b><br>Dashed Line: Entry Point<br>Neon Line: Take Profit / Warning<br>Solid Line: Major Wall",
            showarrow=False,
            font=dict(color="white", size=11),
            align="left",
            bgcolor="rgba(0,0,0,0.6)",
            bordercolor="gray"
        )

        fig.show()

if __name__ == "__main__":
    gen = GoldChartGenerator()
    df = gen.fetch_data(period="5d", interval="5m")
    if df is not None:
        gen.generate_chart(display_hours=12)

--- GC=F データ取得中 ---
